# 7. Classification and Regression Trees

In [2]:
from random import seed, randrange

from functions import *

### Define functions
* Decision trees for classification and regression using Gini index to split

In [3]:
# Calculate the Gini index for a split dataset
# Inputs: list of group dataset rows after node split, list of distinct class values
# Outputs: Gini index, number between 0.0 and 1.0
def cart_gini_index(groups, class_values):
    gini = 0.0
    # Iterate over known class values, to calculate proportion of each in both split groups
    for class_value in class_values:
        # For each known class value, iterate over each group to count occurences
        for group in groups:
            # Check that the group is not empty to avoid zero division
            size = len(group)
            if size == 0:
                continue
            # [...] builds list of classes (observations, not distinct!) in current group
            # Count occurrences of current class in list, and divides by group count of rows
            proportion = [row[-1] for row in group].count(class_value) / float(size)
            # Increments Gini index
            gini += (proportion * (1.0 - proportion))
    return gini

# Split (2-group) a dataset based on an attribute and an attribute value
# Inputs: dataset (or subset), index of column to split by, and column split value
# Outputs: left and right lists of groups of rows (either or both can be empty)
# Note: group 1 if < split value, group 2 if >= split value
def cart_test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            # If row value < threshold, add row to left group
            left.append(row)
        else:
            # If row value >= threshold, add row to right group
            right.append(row)
    return left, right

# NOTE: INEFFICIENT IN THAT IT DOESN'T IGNORE DUPLICATE COLUMN VALUES!
# Determine best split of provided dataset, giving dictionary with best possible split:
# Returns column index to split with, split value, Gini, and tuple with left/right lists
# Minimizes Gini by testing splitting dataset by each column, and each column value
def cart_get_split(dataset):
    # Get distinct class values, assuming class is last column. Converts set to list.
    class_values = list(set(row[-1] for row in dataset))
    # Placeholder best column index to split by, split value, Gini, and resulting groups
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    # Iterating over columns; -1 since last column is the class (output)
    for index in range(len(dataset[0])-1):
        # Iterating over each row (i.e. column value) to test Gini of column + split value
        for row in dataset:
            # Splits groups using current column and current row as split value
            groups = cart_test_split(index, row[index], dataset)
            # Gets Gini Index when ussing current row value as split value
            gini = cart_gini_index(groups, class_values)
            #print('X%d < %.3f Gini = %.3f' % (index+1, row[index], gini))
            # If current Gini is best up to this point, store the split configuration
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    # Return dictionary of best column to split by, split value, and resulting groups tuple
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

# Create terminal node prediction value
# Input: group (list) of rows assigned to the terminal node
# Output: predicted class, based on most common class value in group
def cart_to_terminal(group):
    # Extracts class values in group, assuming class is last column
    outcomes = [row[-1] for row in group]
    # Return highest frequency class value
    return max(set(outcomes), key=outcomes.count)

# Recursively create child splits for a node, or make a terminal node
# Input: pre-split node, max tree depth, minimum rows per node, node's tree depth
# Output: None, i.e. void function
def cart_split(node, max_depth, min_size, depth):
    # Extract left and right lists of group rows from the supplied node (dictionary)
    left, right = node['groups']
    # Deletes groups of data from parent node, as it no longer needs access to the data
    del(node['groups'])
    # Checks whether left or right list empty, i.e. whether a no split (100% in one group)
    if not left or not right:
        # Make the only child a terminal node , and set 'left' and 'right' to point to it
        node['left'] = node['right'] = cart_to_terminal(left + right)
        # Exit current iteration, since terminal child node has no child nodes of its own
        return
    # Check whether supplied node is at or above maximum tree depth
    if depth >= max_depth:
        # Set left and right child nodes to terminal nodes
        node['left'], node['right'] = cart_to_terminal(left), cart_to_terminal(right)
        # Exit current iteration, i.e. halting progression down this branch
        return

    # If we reach this point, we neither have a no split, nor have reached max depth
    # Process left child: if shorter than minimum row size, make it terminal
    if len(left) <= min_size:
        node['left'] = cart_to_terminal(left)
    # Neither too deep nor too small, so split left child node to two child nodes
    else:
        # Split left child node
        node['left'] = cart_get_split(left)
        # Recursively call function on the split left child node in a depth first fashion
        cart_split(node['left'], max_depth, min_size, depth+1)

    # Process right child: if shorter than minimum, make it a terminal node
    if len(right) <= min_size:
        node['right'] = cart_to_terminal(right)
    # If not, split right child node and make a recursive function call
    else:
        # Split right child node
        node['right'] = cart_get_split(right)
        # Make recursive call on the split right child
        cart_split(node['right'], max_depth, min_size, depth+1)

# Build a decision tree
def cart_build_tree(train, max_depth, min_size):
    # Split the root node into two child nodes
    root = cart_get_split(train)
    # Call recursive function to add left nodes then right nodes in a depth first fashion
    cart_split(root, max_depth, min_size, 1)
    # Return root node; now just a dictionary with two child node references
    # Similarly, its child nodes are only references, until terminal nodes which contain rows
    return root

# Print a decision tree, stepping recursively
def print_tree(node, depth=0):
    # If node is a dictionary, i.e. has column index, split value and tuple of groups
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % (depth*' ', (node['index']+1), node['value']))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    # Else node is a terminal node, i.e. contains only class prediction at that node
    else:
        # Multiplies spaces for node indendation
        # Printing "node" is here a list with a single value, i.e. the class prediction
        print('%s[%s]' % (depth*' ', node))

# Recursively make a prediction with a decision tree, given starting node and row of data
def cart_predict(node, row):
    # Determine direction to traverse, using row data and node split column + value
    if row[node['index']] < node['value']:
        # Traversing left, so check whether left child is full node or terminal node
        if isinstance(node['left'], dict):
            # Dictionary, so fully defined node, hence make a recursive call to the left
            return cart_predict(node['left'], row)
        else:
            # Terminal node, so return that node - i.e. the single-value prediction
            return node['left']
    else:
        # Row value >= split value, so traversing right, hence check whether terminal
        if isinstance(node['right'], dict):
            # Fully defined node, so make a recursive call on the right child node
            return cart_predict(node['right'], row)
        else:
            # Terminal node, so return the predicted class
            return node['right']

# Classification and Regression Tree Algorithm
# Train and test are passed by evaluation harness directly, depth and size by *args
# Input: train and test sets, maximum tree depth, minimum rows per non-terminal node
def decision_tree(train, test, max_depth, min_size):
    # 'tree' is the root node of the tree after it is built
    tree = cart_build_tree(train, max_depth, min_size)
    predictions = list()
    # Iterating over test set, running model built on train set and storing predictions
    for row in test:
        prediction = cart_predict(tree, row)
        predictions.append(prediction)
    # Returns the list of prediction for each row in the test set
    return predictions

### Testing CART on contrived dataset

In [4]:
# Test worst possible Gini index of 1.0 due to 50/50 split of classes in each group
# The last [0, 1] is the set of possible classes.
print("Worst:", cart_gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
# This would do the same: print("Worst:", gini_index([[[1], [0]], [[1], [0]]], [0, 1]))

# Test best possible Gini index of 0.0 where group A only contains class 0, and B only 1
print("Best:", cart_gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))

# Test dataset splitting process on contrived dataset: X1, X2, Y
dataset =   [[2.771244718,1.784783929,0],
            [1.728571309,1.169761413,0],
            [3.678319846,2.81281357,0],
            [3.961043357,2.61995032,0],
            [2.999208922,2.209014212,0],
            [7.497545867,3.162953546,1],
            [9.00220326,3.339047188,1],
            [7.444542326,0.476683375,1],
            [10.12493903,3.234550982,1],
            [6.642287351,3.319983761,1]]

split = cart_get_split(dataset)
print('\nTest function for optimal split on contrived dataset:')
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

# Test recursive tree building function on contrived dataset: X1, X2, Y
print('\nTest recursive tree-building using contrived dataset:')
tree = cart_build_tree(dataset, 2, 1)
print_tree(tree)

# Testing prediction using a decision stump (single node)
print('\nTest prediction using decision stump and contrived dataset:')
stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left': 0}
for row in dataset:
    prediction = cart_predict(stump, row)
    print('Expected=%d, Got=%d' % (row[-1], prediction))

Worst: 1.0
Best: 0.0

Test function for optimal split on contrived dataset:
Split: [X1 < 6.642]

Test recursive tree-building using contrived dataset:
[X1 < 6.642]
 [X1 < 2.771]
  [0]
  [0]
 [X1 < 7.498]
  [1]
  [1]

Test prediction using decision stump and contrived dataset:
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


### Testing CART on Banknote dataset

In [5]:
# Load and convert Banknote dataset
seed(1)
dataset = load_csv('data/data_banknote_authentication.csv')
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)

# Train/test split of 0.6
print("\nTesting train/test split CART on Banknote dataset:")
train, test = train_test_split(dataset, 0.6)
tree = cart_build_tree(train, 2, 10)
print_tree(tree)

predictions, actuals = list(), list()
for row in test:
    predictions.append(cart_predict(tree, row))
    actuals.append(row[-1])
print('Train/test accuracy: %.3f%%' % accuracy_metric(actuals, predictions))

# 5 folds gives 274 records in each fold
print("\nEvaluate CART on Banknote dataset using k-fold cross-validation:")
seed(1) # Reset seed since the train/test split above would otherwise change the below
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, accuracy_metric, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))


Loaded data file data/data_banknote_authentication.csv with 1372 rows and 5 columns.

Testing train/test split CART on Banknote dataset:
[X1 < 1.594]
 [X2 < 9.322]
  [1.0]
  [0.0]
 [X3 < -4.786]
  [1.0]
  [0.0]
Train/test accuracy: 81.934%

Evaluate CART on Banknote dataset using k-fold cross-validation:
Scores: [83.57664233576642, 82.84671532846716, 86.86131386861314, 79.92700729927007, 82.11678832116789]
Mean Accuracy: 83.066%
